In [44]:
from lxml import etree
from bs4 import BeautifulSoup
import html5lib
import time
from PIL import Image
import json
import requests

def fill():
    print("----------------------------------------")
    time.sleep(1)
    print("press 1 to Enter your image of driving license")
    print("                  or                        ")
    print("press 2 to Enter data Manually")
    time.sleep(1)
    try:
        press = int(input(">> "))
    except:
        print("try again,(input should be Integer)")
        press = int(input(">> "))
    #     print(press)
    if press == 1:
        print(":: you have choosen for image ::")
        print("@path example:-> D:\file\DL.img")
        DL_path = input("Give path of driving license image here  :-> ")
    else:
        print("")
        print(" :: you have chosen to enter data manually ::")
        print("")
        Dl = input("Enter Driving License No. : ")
        dob = input("Enter your date of birth here(eg.09-02-1997) :  ")
        time.sleep(1)
        print("")
        print(" :: collecting your given data...........! ::")
        print("")


    if press ==1:
        im = Image.open(r"driving license.jpg") 
        #     im = Image.open(DL_path)    

        # Cropped image 
        im1 = im.crop((110, 50, 225, 99)) 
        # im1.show() 
        im1.save("dlcrop.jpg")
#for making it generalized,here we can use yolo(Deep Learning) for card detection and text detection

        img = Image.open('dlcrop.jpg')
        pytesseract.pytesseract.tesseract_cmd ='C:\Program Files\Tesseract-OCR/tesseract.exe'
        result = pytesseract.image_to_string(img) 
        with open('required_data.txt',mode ='w') as file:
            file.write(result)
        #        print(result)   
        file = open('required_data.txt')
        lines = file.readlines()
        DL = lines[0]
        DOB = lines[3].replace("/","-")
    #         print(DL)
    #         print(DOB)
    else :
        DL = Dl
        DOB = dob
    #         print(DL,DOB)
    
    print(" ::Enter the verification code(captcha)::")
    Captcha = input(">>")
    
    DL_no = {'Driving Licence No.':DL} 
    D_birth = {'Date Of Birth':DOB}
    C_aptcha = {'Enter Verification Code':Captcha}
    time.sleep(2)
    
    try:
        data = DL_no + D_birth  + C_aptcha
        response = requests.post("https://parivahan.gov.in/rcdlstatus/?pur_cd=101",data)
        elementid1 = etree.parse.xpath("//input[@id='form_rcdl:tf_dlNO']")         
        response1 = requests.post(elementid1,DL_no)

        elementid2 = etree.xpath("//input[@id='form_rcdl:tf_dob_input']")         
        response2 = requests.post(elementid2,D_birth)

        elementid3 = etree.xpath("//img[@id='form_rcdl:j_idt34:j_idt41']")         
        response3 = requests.post(elementid3,C_aptcha) 
        response = response1+response2+response3
#         print(response.status_code) 
#         print(response.text) 
    except:
        print(" :: yet! for now,unable to solve captcha :: ")
        print(" :: here we are scraping the offline status file(result.html),which loads after filling form successfully ::")
        pass
       
    #result 
    
    myFile=open('result.html','r')
    soup=BeautifulSoup(myFile,"html5lib")
    table = soup.find('table', {'class': 'table table-responsive table-striped table-condensed table-bordered data-table'})
    table_body=table.find('tbody')
    rows = table_body.find_all('tr')
    for row in rows:
        cols=row.find_all('td')
        break
    cols = str(cols)    
    d_expiry = cols[214:225]
    d_issue = cols[137:148]
#     print(d_issue,d_expiry)

    table2 =soup.find('div',{'class' : 'ui-datatable-tablewrapper'})
    table_body=table2.find('tbody')
    rows = table_body.find_all('tr')
    rows = str(rows)
    c_vehicle = rows[119:125]
#     print(c_vehicle)
    print(" :: your result will be saved in json form as :: ")
    time.sleep(2)
    dic = {
                "date of issue" : d_issue,
                "date of expiry" : d_expiry,
                "class of vehicle" : c_vehicle
                }
    print(dic)

    file = open("result.json", "w")  
    json.dump(dic,file,indent = 6)  
    file.close()
    
    
#     def captcha():
#         ""
    
        
    
fill()
# captcha()

----------------------------------------
press 1 to Enter your image of driving license
                  or                        
press 2 to Enter data Manually
>> 2

 :: you have chosen to enter data manually ::

Enter Driving License No. : DL-0420110149646
Enter your date of birth here(eg.09-02-1997) :  09-02-1997

 :: collecting your given data...........! ::

 ::Enter the verification code(captcha)::
>>i don`t know
 :: yet! for now,unable to solve captcha :: 
 :: here we are scraping the offline status file(result.html),which loads after filling form successfully ::
 :: your result will be saved in json form as :: 
{'date of issue': '01-Mar-2011', 'date of expiry': '08-Feb-2026', 'class of vehicle': 'ADPVEH'}
